# Web Scraping Explore

Se extraera información de una página web dedicada a catalogar exoplanetas:

In [14]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns


# Seleccionar el recurso a descargar
resource_url = "https://science.nasa.gov/exoplanets/exoplanet-catalog/"

# Petición para descargar el fichero de Internet
response = requests.get(resource_url, time.sleep(10))

# Si la petición se ha ejecutado correctamente (código 200), entonces el contenido HTML de la página se ha podido descargar
if response:
    # Transformamos el HTML plano en un HTML real (estructurado y anidado, con forma de árbol)
    soup = BeautifulSoup(response.text, 'html')
    soup


In [15]:
# Se verifica response
response

<Response [200]>

In [16]:
# Se encuentran los contenedores de la información
soup.find_all("div", class_="hds-a11y-heading-22")

[<div class="hds-a11y-heading-22">TOI-6695 c</div>,
 <div class="hds-a11y-heading-22">Kepler-139 f</div>,
 <div class="hds-a11y-heading-22">AT2021ueyL b</div>,
 <div class="hds-a11y-heading-22">KMT-2017-BLG-2197L b</div>,
 <div class="hds-a11y-heading-22">KMT-2022-BLG-2076L b</div>,
 <div class="hds-a11y-heading-22">ZTF J1828+2308 b</div>,
 <div class="hds-a11y-heading-22">KMT-2022-BLG-1790L b</div>,
 <div class="hds-a11y-heading-22">KMT-2023-BLG-2209L b</div>,
 <div class="hds-a11y-heading-22">GJ 1289 b</div>,
 <div class="hds-a11y-heading-22">TOI-1453 b</div>,
 <div class="hds-a11y-heading-22">TOI-1453 c</div>,
 <div class="hds-a11y-heading-22">TOI-7041 b</div>,
 <div class="hds-a11y-heading-22">TOI-5143 c</div>,
 <div class="hds-a11y-heading-22">TOI-4364 b</div>,
 <div class="hds-a11y-heading-22">KOI-1843.03</div>]

In [ ]:
# all_stellar_magnitude = soup.find_all("div", class_="CustomField")
# print(all_stellar_magnitude)

items = soup.find_all('div', class_= 'CustomField')
#print(items)

indicador = []
valor = []

# def find_value_and_indicator():
#     val_i = tem_d.find('span', string=lambda t: t and 'Stellar Magnitude:' in t)


    ##Ciclo for para tomar todos los indicadores
for item in items:
    tem_d = item.find('div', class_=False)
    #print(tem_d)
    #tem_v = tem_d.find('span', string=lambda t: t and 'Stellar Magnitude:' in t)
    # tem_v = tem_d.find('span', class_='font-weight-bold')
    # tem_valor = tem_d.find('span', class_=False).text.strip()
    #val_sm = tem_d.find('span', string=lambda t: t and 'Stellar Magnitude:' in t)
    tag_c = tem_d.find('span', class_="font-weight-bold")
    val_name = tag_c.text.strip()
    val_value = tag_c.find_next('span').text.strip()
    obj = {}
    if val_name == 'Stellar Magnitude:':
        obj[val_name] = val_value
    if val_name == 'Stellar Magnitude:':
        obj[val_name] = val_value
    # val_pm = tem_v = tem_d.find('span', string=lambda t: t and 'Planet Mass:' in t)
    # val_pe = tem_v = tem_d.find('span', string=lambda t: t and 'Parsecs from Earth:' in t)
    # val_dd = tem_v = tem_d.find('span', string=lambda t: t and 'Discovery Date:' in t) 

    # if tem_d and val_sm:
    #     team_valor = tem_d.find('span', class_=False).text.strip()
    #     valor.append({
    #         #"Name's Exoplanet" : val.text,
    #         val_sm : team_valor,
    #         "Planet Mass": 0,
    #         "Parsecs from Earth" : 0, 
    #         "Discovery Date": 0
    #     })
    
    # if tem_v != None:
    #     tem_valor = tem_d.find('span', class_=False).text.strip()
    #     if tem_valor != 'Unknown':
    #         valor.append({
    #             "Name's Exoplanet" : tem_v.text,
    #             "Stellar Magnitude" : tem_valor,
    #             "Planet Mass": 20,
    #             "Parsecs from Earth" : tem_valor, 
    #             "Discovery Date": 2025
    #         })
    #     else:
    #         valor.append({
    #             "Name's Exoplanet" : tem_valor,
    #             "Stellar Magnitude" : 0,
    #             "Planet Mass": 20,
    #             "Parsecs from Earth" : tem_valor, 
    #             "Discovery Date": 2025
    #         })

print(valor)

[{'Parsecs from Earth:': '282.536'}, {'Planet Mass:': '1.45 Jupiters'}, {'Stellar Magnitude:': '10.768'}, {'Discovery Date:': '2025'}, {'Parsecs from Earth:': '391.038'}, {'Planet Mass:': '36 Earths'}, {'Stellar Magnitude:': '12.775'}, {'Discovery Date:': '2025'}, {'Parsecs from Earth:': '1040'}, {'Planet Mass:': '1.34 Jupiters'}, {'Stellar Magnitude:': 'Unknown'}, {'Discovery Date:': '2025'}, {'Parsecs from Earth:': '7800'}, {'Planet Mass:': '8.84 Jupiters'}, {'Stellar Magnitude:': 'Unknown'}, {'Discovery Date:': '2025'}, {'Parsecs from Earth:': '5570'}, {'Planet Mass:': '0.88 Jupiters'}, {'Stellar Magnitude:': 'Unknown'}, {'Discovery Date:': '2025'}, {'Parsecs from Earth:': '201.827'}, {'Planet Mass:': '20 Jupiters'}, {'Stellar Magnitude:': '18.0447'}, {'Discovery Date:': '2025'}, {'Parsecs from Earth:': '6480'}, {'Planet Mass:': '1.73 Jupiters'}, {'Stellar Magnitude:': 'Unknown'}, {'Discovery Date:': '2025'}, {'Parsecs from Earth:': '5570'}, {'Planet Mass:': '0.8 Jupiters'}, {'Stell

In [69]:
#valor

In [39]:
soup.find_all("div", class_="hds-a11y-heading-22")[1].text

'Kepler-139 f'